In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
import re
from typing import Dict, List, Tuple
import json

In [2]:
df = pd.read_csv('fake_merchant_dataset.csv')
df

,merchant_id,city,mcc_code,mcc_description,message
0,1,São José dos Campos,5411,"Grocery Stores, Supermarkets",tenho interesse em roupa masculina
1,1,São José dos Campos,5411,"Grocery Stores, Supermarkets",quero divulgar promoções da minha loja
2,1,São José dos Campos,5411,"Grocery Stores, Supermarkets",preciso de alguém que me ajude com rede social
3,1,São José dos Campos,5411,"Grocery Stores, Supermarkets",como funciona?
4,1,São José dos Campos,5411,"Grocery Stores, Supermarkets",quero vender mais
...,...,...,...,...,...
94,20,Santos,5814,Fast Food Restaurants,estou buscando fornecedores de embalagens na m...
95,20,Santos,5814,Fast Food Restaurants,"olá, preciso encontrar parceiros de frete pra ..."
96,20,Santos,5814,Fast Food Restaurants,conhece alguem que vende docinhos?
97,20,Santos,5814,Fast Food Restaurants,procuro fornecedores


In [3]:
# data_analysis_fixed.py
# Versão corrigida que resolve o problema do JSON serialization

import pandas as pd
import numpy as np
from collections import Counter, defaultdict
import re
import json

class MerchantDataAnalyzer:
    def __init__(self, csv_path: str):
        """
        Inicializa o analisador com o dataset de merchants
        """
        self.df = pd.read_csv(csv_path)
        self.analysis_results = {}
        
        print(f"📊 Dataset carregado: {len(self.df)} registros")
        print(f"🏪 Merchants únicos: {self.df['merchant_id'].nunique()}")
        print(f"🏙️ Cidades: {self.df['city'].nunique()}")
        print(f"📝 Categorias MCC: {self.df['mcc_code'].nunique()}")
        
    def basic_statistics(self):
        """
        Estatísticas básicas do dataset
        """
        print("\n" + "="*50)
        print("📈 ESTATÍSTICAS BÁSICAS")
        print("="*50)
        
        print(f"Total de mensagens: {len(self.df)}")
        print(f"Merchants únicos: {self.df['merchant_id'].nunique()}")
        print(f"Mensagens por merchant (média): {len(self.df) / self.df['merchant_id'].nunique():.1f}")
        
        # Distribuição por cidade
        city_dist = self.df['city'].value_counts()
        print(f"\n🏙️ Distribuição por cidade:")
        for city, count in city_dist.items():
            print(f"  {city}: {count} mensagens")
            
        # Distribuição por MCC - CORREÇÃO AQUI
        mcc_dist = self.df.groupby(['mcc_code', 'mcc_description']).size().sort_values(ascending=False)
        print(f"\n🏪 Top 5 categorias MCC:")
        for (mcc_code, desc), count in mcc_dist.head().items():
            print(f"  {mcc_code} - {desc}: {count} mensagens")
            
        # CORREÇÃO: Converter para formato serializável
        mcc_dict = {}
        for (mcc_code, desc), count in mcc_dist.items():
            mcc_dict[f"{mcc_code}_{desc}"] = count
            
        self.analysis_results['basic_stats'] = {
            'total_messages': len(self.df),
            'unique_merchants': self.df['merchant_id'].nunique(),
            'cities': city_dist.to_dict(),
            'mcc_categories': mcc_dict  # Agora serializável
        }
        
    def identify_geographic_clusters(self):
        """
        Identifica clusters geográficos de alta atividade
        """
        print("\n" + "="*50)
        print("🗺️ ANÁLISE DE CLUSTERS GEOGRÁFICOS")
        print("="*50)
        
        # Merchants únicos por cidade
        merchants_by_city = self.df.groupby('city')['merchant_id'].nunique().sort_values(ascending=False)
        
        # Densidade de mensagens por cidade
        messages_by_city = self.df['city'].value_counts()
        
        print("📍 Densidade de merchants por cidade:")
        for city in merchants_by_city.index:
            merchant_count = merchants_by_city[city]
            message_count = messages_by_city[city]
            density = message_count / merchant_count
            
            print(f"  {city}:")
            print(f"    • {merchant_count} merchants únicos")
            print(f"    • {message_count} mensagens totais")
            print(f"    • {density:.1f} mensagens/merchant")
            
            # Identificar hotspots
            if merchant_count >= 3 and density >= 4:
                print(f"    🔥 HOTSPOT IDENTIFICADO!")
                
        # Análise detalhada do cluster Santos
        print(f"\n🔍 ANÁLISE DETALHADA - SANTOS (Principal Cluster)")
        santos_data = self.df[self.df['city'] == 'Santos']
        santos_merchants = santos_data['merchant_id'].unique()
        
        print(f"Merchants em Santos: {[int(x) for x in santos_merchants]}")  # Convert to int
        
        for merchant_id in santos_merchants:
            merchant_data = santos_data[santos_data['merchant_id'] == merchant_id]
            mcc_info = merchant_data.iloc[0]
            messages = merchant_data['message'].tolist()
            
            print(f"\n  Merchant {int(merchant_id)}:")  # Convert to int
            print(f"    • Categoria: {mcc_info['mcc_description']}")
            print(f"    • Mensagens ({len(messages)}):")
            for msg in messages:
                print(f"      - {msg}")
                
        self.analysis_results['geographic_clusters'] = {
            'merchants_by_city': merchants_by_city.to_dict(),
            'messages_by_city': messages_by_city.to_dict(),
            'santos_merchants': [int(x) for x in santos_merchants],  # Convert to serializable
            'hotspots': ['Santos', 'Sorocaba']
        }
        
    def analyze_message_patterns(self):
        """
        Analisa padrões nas mensagens para identificar necessidades
        """
        print("\n" + "="*50)
        print("🔍 ANÁLISE DE PADRÕES NAS MENSAGENS")
        print("="*50)
        
        # Palavras-chave importantes
        keywords = {
            'partnerships': ['parceiro', 'parceria', 'conectar'],
            'suppliers': ['fornecedor', 'fornecedores'],
            'logistics': ['frete', 'entrega', 'campinas'],
            'marketing': ['divulga', 'marketing', 'promoc', 'post', 'insta'],
            'networking': ['conectar', 'conhecer', 'me conectar'],
            'location_based': ['região', 'alguém da', 'da região'],
            'services': ['faço', 'ofereço', 'trabalho com'],
            'products': ['vendo', 'compro', 'atacado', 'varejo']
        }
        
        pattern_counts = defaultdict(int)
        pattern_examples = defaultdict(list)
        
        # Análise de padrões
        for _, row in self.df.iterrows():
            message = row['message'].lower()
            
            for pattern_name, words in keywords.items():
                if any(word in message for word in words):
                    pattern_counts[pattern_name] += 1
                    if len(pattern_examples[pattern_name]) < 3:  # Guardar até 3 exemplos
                        pattern_examples[pattern_name].append({
                            'merchant_id': int(row['merchant_id']),  # Convert to int
                            'city': row['city'],
                            'message': row['message']
                        })
        
        print("📊 Frequência de padrões:")
        for pattern, count in sorted(pattern_counts.items(), key=lambda x: x[1], reverse=True):
            print(f"  {pattern}: {count} ocorrências")
            print("    Exemplos:")
            for example in pattern_examples[pattern]:
                print(f"      • Merchant {example['merchant_id']} ({example['city']}): '{example['message']}'")
            print()
            
        # Necessidades específicas
        specific_needs = {
            'clothing': ['roupa', 'roupas'],
            'electronics': ['eletronic', 'eletron', 'tv'],
            'sweets': ['doce', 'doces', 'confeitaria'],
            'packaging': ['embalagem', 'embalagens'],
            'toys': ['brinquedo', 'brinquedos'],
            'events': ['evento', 'eventos', 'festa'],
            'campinas_delivery': ['campinas']
        }
        
        needs_counts = defaultdict(int)
        needs_examples = defaultdict(list)
        
        for _, row in self.df.iterrows():
            message = row['message'].lower()
            
            for need_name, words in specific_needs.items():
                if any(word in message for word in words):
                    needs_counts[need_name] += 1
                    if len(needs_examples[need_name]) < 2:
                        needs_examples[need_name].append({
                            'merchant_id': int(row['merchant_id']),  # Convert to int
                            'city': row['city'],
                            'message': row['message']
                        })
        
        print("🎯 Necessidades específicas identificadas:")
        for need, count in sorted(needs_counts.items(), key=lambda x: x[1], reverse=True):
            if count > 0:
                print(f"  {need}: {count} menções")
                for example in needs_examples[need]:
                    print(f"    • Merchant {example['merchant_id']} ({example['city']}): '{example['message']}'")
                print()
                
        self.analysis_results['message_patterns'] = {
            'pattern_counts': dict(pattern_counts),
            'specific_needs': dict(needs_counts),
            'pattern_examples': dict(pattern_examples)
        }
        
    def identify_business_opportunities(self):
        """
        Identifica oportunidades de negócio baseadas em padrões complementares
        """
        print("\n" + "="*50)
        print("💡 OPORTUNIDADES DE NEGÓCIO IDENTIFICADAS")
        print("="*50)
        
        opportunities = []
        
        # 1. Santos Logistics Cluster
        santos_logistics = self.df[
            (self.df['city'] == 'Santos') & 
            (self.df['message'].str.lower().str.contains('frete|entrega|parceiro.*frete'))
        ]['merchant_id'].unique()
        
        if len(santos_logistics) >= 3:
            opportunities.append({
                'type': 'logistics_cluster',
                'name': 'Santos Shared Logistics',
                'merchants': [int(x) for x in santos_logistics],  # Convert to int
                'description': f'{len(santos_logistics)} merchants em Santos precisam de frete compartilhado',
                'priority': 'HIGH'
            })
            
        # 2. Campinas Delivery Network
        campinas_delivery = self.df[
            self.df['message'].str.lower().str.contains('campinas')
        ]['merchant_id'].unique()
        
        if len(campinas_delivery) >= 2:
            opportunities.append({
                'type': 'delivery_network',
                'name': 'Campinas Delivery Network',
                'merchants': [int(x) for x in campinas_delivery],  # Convert to int
                'description': f'{len(campinas_delivery)} merchants fazem/precisam entrega em Campinas',
                'priority': 'MEDIUM'
            })
            
        # 3. Marketing Service Providers vs Needers
        marketing_providers = self.df[
            self.df['message'].str.lower().str.contains('faço.*post|trabalho.*marketing|faço.*video')
        ]['merchant_id'].unique()
        
        marketing_needers = self.df[
            self.df['message'].str.lower().str.contains('preciso.*marketing|ajuda.*divulga|preciso.*post')
        ]['merchant_id'].unique()
        
        if len(marketing_providers) > 0 and len(marketing_needers) > 0:
            opportunities.append({
                'type': 'service_matching',
                'name': 'Marketing Services Network',
                'providers': [int(x) for x in marketing_providers],  # Convert to int
                'needers': [int(x) for x in marketing_needers],  # Convert to int
                'description': f'{len(marketing_providers)} providers, {len(marketing_needers)} needers',
                'priority': 'HIGH'
            })
            
        # Continue with other opportunities...
        
        print("🎯 Oportunidades identificadas:")
        for i, opp in enumerate(opportunities, 1):
            print(f"\n{i}. {opp['name']} ({opp['priority']} priority)")
            print(f"   Tipo: {opp['type']}")
            print(f"   Descrição: {opp['description']}")
            if 'merchants' in opp:
                print(f"   Merchants: {opp['merchants']}")
            if 'providers' in opp and 'needers' in opp:
                print(f"   Providers: {opp['providers']}")
                print(f"   Needers: {opp['needers']}")
                
        self.analysis_results['business_opportunities'] = opportunities
        
    def detect_quality_issues(self):
        """
        Detecta mensagens de baixa qualidade que precisam de moderação
        """
        print("\n" + "="*50)
        print("⚠️ DETECÇÃO DE PROBLEMAS DE QUALIDADE")
        print("="*50)
        
        quality_issues = []
        
        # Padrões de baixa qualidade
        low_quality_patterns = [
            (r'^alguém\?$', 'too_vague_question'),
            (r'^quero$', 'incomplete_statement'),
            (r'^oi td bom\?$', 'social_greeting'),
            (r'^como funciona\?$', 'platform_question'),
            (r'^parcerias\?$', 'one_word_inquiry'),
            (r'^conectar.*$', 'vague_connection_request')
        ]
        
        for _, row in self.df.iterrows():
            message = row['message'].lower().strip()
            issues = []
            
            # Check patterns
            for pattern, issue_type in low_quality_patterns:
                if re.match(pattern, message):
                    issues.append(issue_type)
                    
            # Check length
            if len(message) < 10:
                issues.append('too_short')
                
            # Check if too generic
            generic_words = ['alguém', 'quero', 'preciso', 'conectar']
            word_count = len(message.split())
            generic_count = sum(1 for word in generic_words if word in message)
            
            if word_count <= 3 and generic_count > 0:
                issues.append('too_generic')
                
            if issues:
                quality_issues.append({
                    'merchant_id': int(row['merchant_id']),  # Convert to int
                    'city': row['city'],
                    'message': row['message'],
                    'issues': issues
                })
                
        print(f"📊 Problemas de qualidade encontrados: {len(quality_issues)}")
        
        # Agrupar por tipo de problema
        issue_counts = defaultdict(int)
        for item in quality_issues:
            for issue in item['issues']:
                issue_counts[issue] += 1
                
        print("\n📈 Tipos de problemas:")
        for issue_type, count in sorted(issue_counts.items(), key=lambda x: x[1], reverse=True):
            print(f"  {issue_type}: {count} ocorrências")
            
        print("\n📝 Exemplos de mensagens problemáticas:")
        for item in quality_issues[:10]:  # Mostrar primeiros 10
            print(f"  Merchant {item['merchant_id']} ({item['city']}): '{item['message']}'")
            print(f"    Problemas: {', '.join(item['issues'])}")
            
        # Merchants com mais problemas
        merchant_issue_counts = defaultdict(int)
        for item in quality_issues:
            merchant_issue_counts[item['merchant_id']] += 1
            
        problematic_merchants = sorted(merchant_issue_counts.items(), 
                                     key=lambda x: x[1], reverse=True)[:5]
        
        print(f"\n⚠️ Merchants com mais problemas de qualidade:")
        for merchant_id, count in problematic_merchants:
            print(f"  Merchant {merchant_id}: {count} mensagens problemáticas")
            
        self.analysis_results['quality_issues'] = {
            'total_issues': len(quality_issues),
            'issue_types': dict(issue_counts),
            'problematic_merchants': dict(problematic_merchants),
            'examples': quality_issues[:10]
        }
        
    def generate_matching_rules(self):
        """
        Gera regras de matching baseadas na análise
        """
        print("\n" + "="*50)
        print("🎯 REGRAS DE MATCHING GERADAS")
        print("="*50)
        
        rules = []
        
        # Regra 1: Santos Logistics Cluster
        santos_logistics = ['4', '6', '10', '15', '20']
        rules.append({
            'name': 'santos_logistics_cluster',
            'description': 'Merchants em Santos que precisam de frete compartilhado',
            'members': santos_logistics,
            'trigger_keywords': ['frete', 'entrega', 'parceiro', 'logística'],
            'weight': 0.9,
            'geographic_constraint': 'Santos'
        })
        
        # Continue with other rules...
        
        print("📋 Regras de matching criadas:")
        for i, rule in enumerate(rules, 1):
            print(f"\n{i}. {rule['name']} (peso: {rule['weight']})")
            print(f"   Descrição: {rule['description']}")
            print(f"   Palavras-chave: {rule['trigger_keywords']}")
            if 'members' in rule:
                print(f"   Membros: {rule['members']}")
                
        self.analysis_results['matching_rules'] = rules
        
    def save_analysis_results(self, output_file='analysis_results.json'):
        """
        Salva todos os resultados da análise em um arquivo JSON
        """
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(self.analysis_results, f, indent=2, ensure_ascii=False)
            
        print(f"\n💾 Resultados salvos em: {output_file}")
        
    def run_full_analysis(self):
        """
        Executa toda a análise de dados
        """
        print("🚀 INICIANDO ANÁLISE COMPLETA DOS DADOS")
        print("="*60)
        
        self.basic_statistics()
        self.identify_geographic_clusters()
        self.analyze_message_patterns()
        self.identify_business_opportunities()
        self.detect_quality_issues()
        self.generate_matching_rules()
        
        print("\n" + "="*60)
        print("✅ ANÁLISE COMPLETA FINALIZADA!")
        print("="*60)
        
        # Summary
        print(f"\n📊 RESUMO EXECUTIVO:")
        print(f"• {self.analysis_results['basic_stats']['unique_merchants']} merchants ativos")
        print(f"• {len(self.analysis_results['geographic_clusters']['hotspots'])} hotspots geográficos identificados")
        print(f"• {len(self.analysis_results['business_opportunities'])} oportunidades de negócio")
        print(f"• {self.analysis_results['quality_issues']['total_issues']} mensagens precisam melhoria")
        print(f"• {len(self.analysis_results['matching_rules'])} regras de matching criadas")
        
        self.save_analysis_results()
        
        return self.analysis_results



In [9]:
# ==================== FUNÇÃO PRINCIPAL ====================

def main():
    """
    Script principal para executar a análise
    """
    # Inicializar analisador
    analyzer = MerchantDataAnalyzer('fake_merchant_dataset.csv')
    
    # Executar análise completa
    results = analyzer.run_full_analysis()
    
    # Insights específicos para implementação
    print("\n🎯 INSIGHTS PARA IMPLEMENTAÇÃO:")
    print("-" * 40)
    
    # Santos cluster insights
    santos_merchants = results['geographic_clusters']['santos_merchants']
    print(f"✅ Santos Logistics Cluster: {len(santos_merchants)} merchants")
    print(f"   IDs: {santos_merchants}")
    print(f"   Estratégia: Priority matching para frete compartilhado")
    
    # Top business opportunity
    if results['business_opportunities']:
        top_opp = results['business_opportunities'][0]
        print(f"\n✅ Principal Oportunidade: {top_opp['name']}")
        print(f"   Tipo: {top_opp['type']}")
        print(f"   Merchants: {top_opp['merchants']}")
    
    # Moderation targets
    quality_issues = results['quality_issues']['total_issues']
    print(f"\n⚠️ Moderação: {quality_issues} mensagens de baixa qualidade")
    print(f"   Merchants problemáticos: {list(results['quality_issues']['problematic_merchants'].keys())}")
    
    # Matching rules count
    rules_count = len(results['matching_rules'])
    print(f"\n🎯 {rules_count} regras de matching prontas para implementação")
    
    return results

if __name__ == "__main__":
    # Para executar: python data_analysis_fixed.py
    results = main()

📊 Dataset carregado: 99 registros
🏪 Merchants únicos: 20
🏙️ Cidades: 8
📝 Categorias MCC: 9
🚀 INICIANDO ANÁLISE COMPLETA DOS DADOS

📈 ESTATÍSTICAS BÁSICAS
Total de mensagens: 99
Merchants únicos: 20
Mensagens por merchant (média): 5.0

🏙️ Distribuição por cidade:
  Santos: 34 mensagens
  Sorocaba: 30 mensagens
  Bauru: 10 mensagens
  São José dos Campos: 5 mensagens
  São Carlos: 5 mensagens
  Ribeirão Preto: 5 mensagens
  São Paulo: 5 mensagens
  Jundiaí: 5 mensagens

🏪 Top 5 categorias MCC:
  5814 - Fast Food Restaurants: 26 mensagens
  7299 - Misc. Personal Services: 23 mensagens
  5945 - Hobby, Toy, and Game Shops: 15 mensagens
  7011 - Hotels and Motels: 10 mensagens
  5411 - Grocery Stores, Supermarkets: 5 mensagens

🗺️ ANÁLISE DE CLUSTERS GEOGRÁFICOS
📍 Densidade de merchants por cidade:
  Santos:
    • 7 merchants únicos
    • 34 mensagens totais
    • 4.9 mensagens/merchant
    🔥 HOTSPOT IDENTIFICADO!
  Sorocaba:
    • 6 merchants únicos
    • 30 mensagens totais
    • 5.0 mensa